In [1]:
import torch
from torch.utils import data
import torchvision
from torchvision import transforms
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

from data import LoadTest, LoadTest2
from model import VGG16
from utils import *

for name in (torch, torchvision, cv2, np):
    print(name.__version__)

1.6.0+cu101
0.7.0+cu101
4.3.0
1.19.1


In [2]:
if torch.cuda.is_available():
    device = torch.device(device='cuda')
else:
    device = torch.device(device='cpu')
device

device(type='cuda')

In [3]:
path_image = "./DUTS/DUTS-TE/DUTS-TE-Image/"
path_mask = "./DUTS/DUTS-TE/DUTS-TE-Mask/"

# path_image = "./DUT-OMROM/DUT-OMRON-image/"
# path_mask = "./DUT-OMROM/pixelwiseGT-new-PNG/"

batch_size = 8 #受限于贫穷，4是极限了
target_size = 256

In [4]:
data_loader = data.DataLoader(LoadTest(path_image, path_mask, target_size),
                            batch_size=batch_size,
                            shuffle=False)

In [5]:
total_batch = len(data_loader)
total_batch

628

In [6]:
model = VGG16()
model.load_state_dict(torch.load("./model/PFA_101.pth"), strict=False)

<All keys matched successfully>

In [7]:
model.eval()
model.to(device)
print()

In [8]:
for layer in model.parameters():
    layer.requires_grad = False

In [9]:

start_time = time.time()

total_loss = 0
total_acc = 0
total_pre = 0
total_rec = 0
total_f_score = 0
mae_list = []
for batch_n, (image, mask) in enumerate(data_loader, start=1):

    image = image.to(device)
    mask = mask.to(device)

    predict = model(image)

    with torch.no_grad():
        acc = accuracy(predict, mask)
        total_acc += acc

        pre = precision(predict, mask)
        total_pre += pre

        rec = recall(predict, mask)
        total_rec += rec

        f_score = F_Measure(pre, rec)
        total_f_score += f_score
        
        ae = torch.mean(torch.abs(predict - mask), dim=(1, 2, 3)).cpu().numpy()
        mae_list.extend(ae)

    if batch_n % 100 == 0:
        with torch.no_grad():
            avg_acc = total_acc / batch_n
            avg_pre = total_pre / batch_n
            avg_rec = total_rec / batch_n
            avg_f_score = total_f_score / batch_n
            print("Batch:{}/{}".format( batch_n, total_batch), end="")
            print(" acc:{} pre:{} recall:{} F-measure:{} MAE:{}"
                  .format(avg_acc, avg_pre, avg_rec, avg_f_score, np.mean(mae_list)))
end_time = time.time()
print("--------------------------------------------------------------")
print("time:{:.2f}s END : acc:{} pre:{} rec:{} F-measure:{} MAE:{}"
      .format(end_time - start_time, 
              total_acc / total_batch,
              total_pre / total_batch,
              total_rec / total_batch,
              total_f_score / total_batch,
             np.mean(mae_list)))
print("--------------------------------------------------------------")

Batch:100/628 acc:0.9270908832550049 pre:0.8718537092208862 recall:0.8050254583358765 F-measure:0.8526804447174072 MAE:0.03669100999832153
Batch:200/628 acc:0.9226404428482056 pre:0.8682411909103394 recall:0.8020406365394592 F-measure:0.848583459854126 MAE:0.03948577493429184
Batch:300/628 acc:0.9195829629898071 pre:0.8750004172325134 recall:0.7969251871109009 F-measure:0.8523285388946533 MAE:0.04117202013731003
Batch:400/628 acc:0.9185425639152527 pre:0.8760495781898499 recall:0.789220929145813 F-measure:0.850858211517334 MAE:0.041825197637081146
Batch:500/628 acc:0.9202502965927124 pre:0.856650173664093 recall:0.7606211304664612 F-measure:0.8274118900299072 MAE:0.04545191302895546
Batch:600/628 acc:0.9218831658363342 pre:0.8456729054450989 recall:0.7450509071350098 F-measure:0.814698338508606 MAE:0.04715414717793465
--------------------------------------------------------------
time:189.07s END : acc:0.9224641919136047 pre:0.8427335023880005 rec:0.742087721824646 F-measure:0.81109797

In [10]:
path_image = "./DUT-OMROM/DUT-OMRON-image/"
path_mask = "./DUT-OMROM/pixelwiseGT-new-PNG/"

data_loader = data.DataLoader(LoadTest(path_image, path_mask, target_size),
                            batch_size=batch_size,
                            shuffle=False)


In [11]:
total_batch = len(data_loader)
total_batch

646

In [12]:

start_time = time.time()

total_loss = 0
total_acc = 0
total_pre = 0
total_rec = 0
total_f_score = 0
mae_list = []
for batch_n, (image, mask) in enumerate(data_loader, start=1):

    image = image.to(device)
    mask = mask.to(device)

    predict = model(image)

    with torch.no_grad():
        acc = accuracy(predict, mask)
        total_acc += acc

        pre = precision(predict, mask)
        total_pre += pre

        rec = recall(predict, mask)
        total_rec += rec

        f_score = F_Measure(pre, rec)
        total_f_score += f_score
        
        ae = torch.mean(torch.abs(predict - mask), dim=(1, 2, 3)).cpu().numpy()
        mae_list.extend(ae)

    if batch_n % 100 == 0:
        with torch.no_grad():
            avg_acc = total_acc / batch_n
            avg_pre = total_pre / batch_n
            avg_rec = total_rec / batch_n
            avg_f_score = total_f_score / batch_n
            print("Batch:{}/{}".format( batch_n, total_batch), end="")
            print(" acc:{} pre:{} recall:{} F-measure:{} MAE:{}"
                  .format(avg_acc, avg_pre, avg_rec, avg_f_score, np.mean(mae_list)))
end_time = time.time()
print("--------------------------------------------------------------")
print("time:{:.2f}s END : acc:{} pre:{} rec:{} F-measure:{} MAE:{}"
      .format(end_time - start_time, 
              total_acc / total_batch,
              total_pre / total_batch,
              total_rec / total_batch,
              total_f_score / total_batch,
             np.mean(mae_list)))
print("--------------------------------------------------------------")

Batch:100/646 acc:0.9240576028823853 pre:0.8152578473091125 recall:0.6763027310371399 F-measure:0.7681876420974731 MAE:0.059682611376047134
Batch:200/646 acc:0.9202075004577637 pre:0.812114417552948 recall:0.6716572642326355 F-measure:0.7648613452911377 MAE:0.06325583904981613
Batch:300/646 acc:0.9243022799491882 pre:0.8117291927337646 recall:0.6850655674934387 F-measure:0.7694783210754395 MAE:0.05963006243109703
Batch:400/646 acc:0.9247965216636658 pre:0.8151596784591675 recall:0.6838058829307556 F-measure:0.7718009948730469 MAE:0.05911847949028015
Batch:500/646 acc:0.9273542761802673 pre:0.8179358839988708 recall:0.7016883492469788 F-measure:0.7795796394348145 MAE:0.05634082853794098
Batch:600/646 acc:0.929520308971405 pre:0.821723222732544 recall:0.7058323621749878 F-measure:0.7836914658546448 MAE:0.054446447640657425
--------------------------------------------------------------
time:192.97s END : acc:0.9296713471412659 pre:0.8196208477020264 rec:0.7045495510101318 F-measure:0.7816

In [13]:
# path_img_mask = "./PASCAL-S/Imgs/"
path_img_mask = "./HKU-IS/Imgs/"
data_loader = data.DataLoader(LoadTest2(path_img_mask, target_size),
                            batch_size=batch_size,
                            shuffle=False)
total_batch = len(data_loader)
total_batch

556

In [14]:

start_time = time.time()

total_loss = 0
total_acc = 0
total_pre = 0
total_rec = 0
total_f_score = 0
mae_list = []
for batch_n, (image, mask) in enumerate(data_loader, start=1):

    image = image.to(device)
    mask = mask.to(device)

    predict = model(image)

    with torch.no_grad():
        acc = accuracy(predict, mask)
        total_acc += acc

        pre = precision(predict, mask)
        total_pre += pre

        rec = recall(predict, mask)
        total_rec += rec

        f_score = F_Measure(pre, rec)
        total_f_score += f_score
        
        ae = torch.mean(torch.abs(predict - mask), dim=(1, 2, 3)).cpu().numpy()
        mae_list.extend(ae)

    if batch_n % 100 == 0:
        with torch.no_grad():
            avg_acc = total_acc / batch_n
            avg_pre = total_pre / batch_n
            avg_rec = total_rec / batch_n
            avg_f_score = total_f_score / batch_n
            print("Batch:{}/{}".format( batch_n, total_batch), end="")
            print(" acc:{} pre:{} recall:{} F-measure:{} MAE:{}"
                  .format(avg_acc, avg_pre, avg_rec, avg_f_score, np.mean(mae_list)))
end_time = time.time()
print("--------------------------------------------------------------")
print("time:{:.2f}s END : acc:{} pre:{} rec:{} F-measure:{} MAE:{}"
      .format(end_time - start_time, 
              total_acc / total_batch,
              total_pre / total_batch,
              total_rec / total_batch,
              total_f_score / total_batch,
             np.mean(mae_list)))
print("--------------------------------------------------------------")

Batch:100/556 acc:0.9304931163787842 pre:0.931165874004364 recall:0.8156136870384216 F-measure:0.9001742601394653 MAE:0.0425398014485836
Batch:200/556 acc:0.9321348071098328 pre:0.9258958101272583 recall:0.7997215986251831 F-measure:0.8905749320983887 MAE:0.04274224117398262
Batch:300/556 acc:0.9339446425437927 pre:0.928520143032074 recall:0.8057048320770264 F-measure:0.8943082094192505 MAE:0.04139125347137451
Batch:400/556 acc:0.9347918629646301 pre:0.9267739057540894 recall:0.8031246662139893 F-measure:0.8920859098434448 MAE:0.04115818813443184
Batch:500/556 acc:0.9359866380691528 pre:0.923620879650116 recall:0.808220386505127 F-measure:0.8912844657897949 MAE:0.04028189927339554
--------------------------------------------------------------
time:173.90s END : acc:0.937131404876709 pre:0.9185669422149658 rec:0.799027144908905 F-measure:0.8838539719581604 MAE:0.04009190574288368
--------------------------------------------------------------


In [15]:
path_img_mask = "./PASCAL-S/Imgs/"
data_loader = data.DataLoader(LoadTest2(path_img_mask, target_size),
                            batch_size=batch_size,
                            shuffle=False)
total_batch = len(data_loader)
total_batch

107

In [16]:

start_time = time.time()

total_loss = 0
total_acc = 0
total_pre = 0
total_rec = 0
total_f_score = 0
mae_list = []
for batch_n, (image, mask) in enumerate(data_loader, start=1):

    image = image.to(device)
    mask = mask.to(device)

    predict = model(image)

    with torch.no_grad():
        acc = accuracy(predict, mask)
        total_acc += acc

        pre = precision(predict, mask)
        total_pre += pre

        rec = recall(predict, mask)
        total_rec += rec

        f_score = F_Measure(pre, rec)
        total_f_score += f_score
        
        ae = torch.mean(torch.abs(predict - mask), dim=(1, 2, 3)).cpu().numpy()
        mae_list.extend(ae)

    if batch_n % 100 == 0:
        with torch.no_grad():
            avg_acc = total_acc / batch_n
            avg_pre = total_pre / batch_n
            avg_rec = total_rec / batch_n
            avg_f_score = total_f_score / batch_n
            print("Batch:{}/{}".format( batch_n, total_batch), end="")
            print(" acc:{} pre:{} recall:{} F-measure:{} MAE:{}"
                  .format(avg_acc, avg_pre, avg_rec, avg_f_score, np.mean(mae_list)))
end_time = time.time()
print("--------------------------------------------------------------")
print("time:{:.2f}s END : acc:{} pre:{} rec:{} F-measure:{} MAE:{}"
      .format(end_time - start_time, 
              total_acc / total_batch,
              total_pre / total_batch,
              total_rec / total_batch,
              total_f_score / total_batch,
             np.mean(mae_list)))
print("--------------------------------------------------------------")

Batch:100/107 acc:0.927498459815979 pre:0.8775147199630737 recall:0.7317379117012024 F-measure:0.8340713977813721 MAE:0.06852822750806808
--------------------------------------------------------------
time:36.47s END : acc:0.9261363744735718 pre:0.8745588064193726 rec:0.7291088700294495 F-measure:0.8307358622550964 MAE:0.06971210986375809
--------------------------------------------------------------
